# Load - Wrangle - EDA

In [1]:
import os
import sys
import pandas as pd
import numpy as np


from wordcloud import WordCloud
from pandas_profiling import ProfileReport

# Visualize
import seaborn as sns
import matplotlib.pyplot as plt
import plotly as plty

import missingno as msno

# Textacy

import spacy
from spacy.matcher import Matcher
from spacy.lang.en.stop_words import STOP_WORDS as stopwords
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

#https://realpython.com/natural-language-processing-spacy-python/
matcher = Matcher(nlp.vocab)

join_comments = True
join_raw_article = True

# Set Project Root
PROJ_ROOT = os.path.join(os.pardir)
print(os.path.abspath(PROJ_ROOT))

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(PROJ_ROOT, "src")
sys.path.append(src_dir)


# %run "$BASE_DIR/settings.py"
from wordcloud import WordCloud

%config InlineBackend.figure_format = 'png'

# Set to Reload all custom packages
%load_ext autoreload
%autoreload 2

%matplotlib inline

# !python -m spacy download en_core_web_sm

from collections import Counter
import pprint
pp = pprint.PrettyPrinter(depth=6)

g:\My Drive\Code\springboard_proj_nlp_nytimes


## Load All Article Comments

In [ ]:
# df_comments = pd.read_csv('..//data//raw//nyt-comments-2020.csv',parse_dates=['createDate','updateDate','approveDate'])
# df_comments.sort_values(by='createDate',inplace=True,ascending=True)

# ## Full Article Verbose / Write to file
# df_comments.to_csv('..//data//processed//df_comments.csv')
# df_comments.to_pickle('..//data//processed//df_comments.pkl')

# ## Sample Article Verbose / Write to file
# # df_sample = df_comments.sample(frac=0.05)
# # df_sample.to_csv('..//data//processed//df_comments_sample.csv')
# # df_sample.to_pickle('..//data//processed//df_comments_sample.pkl')

## Load NYT Articles
Will join with a more verbose set of data
(This is the whole list (not train and test split)

In [2]:
# # All Articles
df_articles = pd.read_csv('..//data//raw//nyt-articles-2020.csv',parse_dates=['pub_date'])
df_articles.sort_values(by='pub_date',inplace=True,ascending=True)

In [ ]:
df_articles

## Load Enhanced Articles

Pull print section, print_page, lead_paragraph, and headlines to provide possible more text features that are not present in original dataset

In [8]:
# build file list matching the pickle file name pattern
filelist = ['..//data//raw//2020-'+ str(x).zfill(2) + '-articles.pickle' for x in range(1,13)]

df_articles_verbose = pd.DataFrame()
df_temp = pd.DataFrame()

# For each one of the file names
for filename in filelist[0:13]:
    print('Filename: ' + filename)
    
    # Get large articles file
    df_json = pd.read_pickle(filename)
    df_json = df_json[['uri','print_section','print_page','lead_paragraph','headline.main','byline.person','byline.organization','headline.sub','text','web_url']]
    
    df_temp = df_temp.append(df_json,ignore_index=True)
 
  
    del(df_json)


Filename: ..//data//raw//2020-01-articles.pickle
Filename: ..//data//raw//2020-02-articles.pickle
Filename: ..//data//raw//2020-03-articles.pickle
Filename: ..//data//raw//2020-04-articles.pickle
Filename: ..//data//raw//2020-05-articles.pickle
Filename: ..//data//raw//2020-06-articles.pickle
Filename: ..//data//raw//2020-07-articles.pickle
Filename: ..//data//raw//2020-08-articles.pickle
Filename: ..//data//raw//2020-09-articles.pickle
Filename: ..//data//raw//2020-10-articles.pickle
Filename: ..//data//raw//2020-11-articles.pickle
Filename: ..//data//raw//2020-12-articles.pickle


## Join enhanced article data to original article df

In [9]:
# Merge Articles Back
df_articles_verbose = pd.merge(df_articles, df_temp, how='left', left_on='uniqueID',right_on='uri',suffixes=['_left','_right'])

In [10]:
df_articles_verbose.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16787 entries, 0 to 16786
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   newsdesk             16787 non-null  object             
 1   section              16787 non-null  object             
 2   subsection           5693 non-null   object             
 3   material             16787 non-null  object             
 4   headline             16787 non-null  object             
 5   abstract             16784 non-null  object             
 6   keywords             16786 non-null  object             
 7   word_count           16787 non-null  int64              
 8   pub_date             16787 non-null  datetime64[ns, UTC]
 9   n_comments           16787 non-null  int64              
 10  uniqueID             16787 non-null  object             
 11  uri                  16787 non-null  object             
 12  print_section     

In [12]:
df_articles_verbose.drop(columns=['headline.sub','byline.organization','subsection'],inplace=True)

In [13]:
## Full Article Verbose / Write to file
df_articles_verbose.to_csv('..//data//processed//df_articles_all.csv')
df_articles_verbose.to_pickle('..//data//processed//df_articles_all.pkl')

# ## Sample Article Verbose / Write to file
# df_sample = df_articles_verbose.sample(frac=0.05)
# df_sample.to_csv('..//data//processed//df_articles_sample.csv')
# df_sample.to_pickle('..//data//processed//df_articles_sample.pkl')


# Data Wrangling - Entry point for review (skips raw loading)

### Load Sample or Full Set of Articles

In [14]:
df_articles = pd.read_pickle('..//data//processed//df_articles_all.pkl')

# # Drop constant columns, userTitle is 99% empty
# df_comments.drop(columns=['status','trusted','recommendedFlag','isAnonymous','userTitle'],inplace=True)

# Convert Timestamps where necessary
df_articles['pub_date'] = pd.to_datetime(df_articles['pub_date'])
# df_comments['createDate'] = pd.to_datetime(df_comments['createDate'])
# df_comments['updateDate'] = pd.to_datetime(df_comments['updateDate'])
# df_comments['approveDate'] = pd.to_datetime(df_comments['approveDate'])

# NAN Drops
# Drop Keyword Columns that are not str
df_articles.dropna(subset=['keywords'],inplace=True) # Key for teh NER to work

In [15]:
df_articles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16786 entries, 0 to 16786
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   newsdesk        16786 non-null  object             
 1   section         16786 non-null  object             
 2   material        16786 non-null  object             
 3   headline        16786 non-null  object             
 4   abstract        16783 non-null  object             
 5   keywords        16786 non-null  object             
 6   word_count      16786 non-null  int64              
 7   pub_date        16786 non-null  datetime64[ns, UTC]
 8   n_comments      16786 non-null  int64              
 9   uniqueID        16786 non-null  object             
 10  uri             16786 non-null  object             
 11  print_section   11597 non-null  object             
 12  print_page      11597 non-null  object             
 13  lead_paragraph  16786 non-null 

In [17]:
df_articles.iloc[20].T

newsdesk                                                      Metro
section                                                    New York
material                                                       News
headline          More People Are Dying on New York City’s Stree...
abstract          More cyclists were killed last year than in an...
keywords          ['Traffic Accidents and Safety', 'Bicycles and...
word_count                                                     1186
pub_date                                  2020-01-01 18:19:38+00:00
n_comments                                                      267
uniqueID          nyt://article/edd3d743-b68d-50ee-9af9-b6ee1200...
uri               nyt://article/edd3d743-b68d-50ee-9af9-b6ee1200...
print_section                                                     A
print_page                                                       16
lead_paragraph    As New York City tackled the stubborn problem ...
headline.main     More People Are Dying on New Y

## Feature Engineering

### Simple Features

#### Add Month and drop unused columns

In [19]:
# Features
df_articles['month'] = df_articles['pub_date'].dt.month

In [20]:
df_articles.describe().T

,count,mean,std,min,25%,50%,75%,max
word_count,16786.0,1300.961813,944.801861,0.0,874.0,1183.0,1502.0,15619.0
n_comments,16786.0,297.056714,513.419176,1.0,21.0,87.0,323.0,8987.0
month,16786.0,6.320743,3.437056,1.0,3.0,6.0,9.0,12.0


## Complex Text Features


### Spacy NER
for ent in piano_class_doc.ents:
...     print(ent.text, ent.start_char, ent.end_char,
...           ent.label_, spacy.explain(ent.label_))

In [21]:
# Get part of speech 
def get_pos_tag_len(text, pos_type):
    doc = nlp(text)
    pos = []
    for token in doc:
        if token.pos_ == pos_type:
            pos.append(token.pos_)
    pos = Counter(pos)
    return (pos[pos_type])


#'PROPN', 'PUNCT', 'DET', 'NOUN', 'VERB', 'ADV', 'SCONJ', 'PRON', 'ADP', 'ADJ', 'AUX', 'NUM', 'INTJ#'

In [22]:
# # Get Entities
def get_ent_tag_len(text,ent_label):
    ents = []
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == ent_label:
            ents.append(ent.text)

    ents = Counter(ents)
    return (len(ents))

# 'PERSON'
# 'NORP' - Nationalities, relgious, political groups
# 'FAC' - Buildings, airports, highways, bridges, etc
# 'ORG' - Companies, Agencies, Institutions
# 'GPE' - Countries, Cities, States
# 'PRODUCT' - Objects, vehicles, foods, etc



### Make POS Features
Count parts of speech by type (POS)

In [ ]:
# Add spacy Nouns
df_articles['TEXT_Keywords_POS_NOUN'] = df_articles['keywords'].apply(get_pos_tag_len,pos_type='NOUN')

In [ ]:
# Add PropN
df_articles['TEXT_Keywords_POS_PNOUN'] = df_articles['keywords'].apply(get_pos_tag_len,pos_type='PROPN')

### Make Ent Labels

In [ ]:
# Add Spacy Entities of Organizations
df_articles['TEXT_Keywords_ENT_ORG'] = df_articles['keywords'].apply(get_ent_tag_len,ent_label='ORG')

In [ ]:
# Add Spacy Entities of Nationalities, Religious, Political Group
df_articles['TEXT_Keywords_ENT_NORP'] = df_articles['keywords'].apply(get_ent_tag_len,ent_label='NORP')

In [ ]:
# Add Spacy Entities Buildings, Airports, Highways
df_articles['TEXT_Keywords_ENT_FAC'] = df_articles['keywords'].apply(get_ent_tag_len,ent_label='FAC')


In [ ]:
# Add Spacy Entities Countries, Cities, States
df_articles['TEXT_Keywords_ENT_GPE'] = df_articles['keywords'].apply(get_ent_tag_len,ent_label='GPE')

In [ ]:
# Add spacy entities Locations
df_articles['TEXT_Keywords_ENT_LOC'] = df_articles['keywords'].apply(get_ent_tag_len,ent_label='LOC')

In [ ]:
# Add spacy entities locations People
df_articles['TEXT_Keywords_ENT_PERSON'] = df_articles['keywords'].apply(get_ent_tag_len,ent_label='PERSON')

### Comment Split to Binary Column

In [ ]:
comment_split = 350
cut_labels = [0,1]
cut_bins = [0,comment_split,max(df_articles['n_comments'])]

# Process Square Footage
# Enable Hue of building size
df_articles['comment_isHigh'] = pd.cut(df_articles['n_comments'], bins=cut_bins, labels=cut_labels)
df_articles.head(1).T

In [ ]:
df_articles.sort_values(by='n_comments',ascending=False,inplace=True)

## Write Engineered Features back to File

In [ ]:
## Full Article Verbose / Write to file
df_articles.to_csv('..//data//processed//df_articles_fe.csv')
df_articles.to_pickle('..//data//processed//df_articles_fe.pkl')

### Reload

In [ ]:
## Load exit from EDA
df_articles = pd.read_pickle('..//data//processed//df_articles_fe.pkl')

# EDA
Entry point for ad-hoc analysis

In [ ]:
df_articles = pd.read_pickle('..//data//processed//df_articles_fe.pkl')

### Drop poorly correlated columns -
df_articles = df_articles.drop(columns=(df_articles.filter(like='LeadParagraph',axis=1).columns))
df_articles = df_articles.drop(columns=(df_articles.filter(like='headline',axis=1).columns))


In [ ]:
df_articles.info()

### Generate Pandas Profiles for EDA - Initial
Create generic EDA, drill down where necesary

EDA Ideas
- Ratio of has n_comments to no n_comments
- Missing data
- Top Words
- Top POS Speech
- Correlations
- Wordcloud
- Comments by Month
- Correlations

In [ ]:
profile_articles = ProfileReport(df_articles, title="New York Times - Articles " + EDA_note)
profile_articles.to_file('..//reports//figures//EDA_Articles' + EDA_note + '.html')

### Examine Missing Data

In [ ]:
msno.matrix(df_articles)

## Jointplot

In [ ]:
sns.jointplot(data=df_articles,x='n_comments',y='word_count');
plt.subplots_adjust(top=0.9)
plt.suptitle('Word Count and n_comments', fontsize = 16);

In [ ]:
plt.ioff()
plt.figure(figsize=(20, 10))

heatmap = sns.heatmap(df_articles.corr(),vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Correlations', fontdict={'fontsize':18}, pad=16);


The strongest correlations (still pretty weak) is the keywords and whether the article speaks of a person (TEXT_Keywords_ENT_PERSON)


In [ ]:
sns.jointplot(data=df_articles,x='n_comments',y='TEXT_Keywords_ENT_PERSON')
plt.subplots_adjust(top=0.9)
plt.suptitle('Word Count and People Entities', fontsize = 16);

In [ ]:
df_articles.head(1).T

## Count all Keywords

In [ ]:
all_keywords = []
for _ in df_articles['keywords'].apply(eval):
    all_keywords.extend(_)
cnt = Counter(all_keywords)
# pp.pprint(cnt.most_common(10))

df_count = pd.DataFrame.from_dict(cnt, orient='index').reset_index().sort_values(by=0,ascending=False)
df_count = df_count.rename(columns={'index':'phrase', 0:'count'}).reset_index(drop=True)
df_count

cnt.most_common(6)

In [ ]:
plt.rcParams['figure.figsize']=10,10
sns.set_theme(style="darkgrid")
df_count = df_count.head(10)
p = sns.barplot(data=df_count,x='count',y='phrase')
p.set_title('Most Common Keywords')
df_count

## Try Sample Spacy NER visualization

In [ ]:
doc = nlp(all_keywords[7])
displacy.render(doc,style='ent')

In [ ]:
doc = nlp(all_keywords[7])
displacy.render(doc,style='ent')

In [ ]:
doc = nlp('Coronavirus (2019-nCoV)')
displacy.render(doc,style='ent')

### WordCloud 

In [ ]:
# Wordcloud
plt.rcParams['figure.figsize']=40,20
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
wordcloud = WordCloud().generate('|'.join(all_keywords))
plt.figure()


### Word Count Distribution

In [ ]:
plt.rcParams['figure.figsize']=(10,8)
hist = sns.histplot(data=df_articles,x='word_count', bins=30, kde=True)
hist.set_title('Word Count Distribution', fontdict={'fontsize':14}, pad=16);

### N_Comments Distribution

In [ ]:
plt.rcParams['figure.figsize']=(10,8)
hist2 = sns.histplot(data=df_articles,x='n_comments', bins=50,kde=True)
hist2.set_title('N Comments Distribution', fontdict={'fontsize':14}, pad=16);

In [ ]:
### 

In [ ]:
plt.rcParams['figure.figsize']=(10,8)
hist2 = sns.histplot(data=df_articles,x='n_comments', bins=50,kde=True)
hist2.set_title('N Comments Distribution', fontdict={'fontsize':14}, pad=16);